In [1]:
#pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

In [2]:
pip install -q gradio

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 212 kB 49.6 MB/s 
     |████████████████████████████████| 1.1 MB 44.9 MB/s 
     |████████████████████████████████| 253 kB 48.6 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 2.0 MB 48.8 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 271 kB 58.2 MB/s 
     |████████████████████████████████| 144 kB 44.9 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 10.9 MB 47.1 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 79 kB 6.3 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 856 kB 51.8 MB/s 
     |████████████████████████████████| 4.0 MB 40.1 MB/s 
     |██████████████████████

In [3]:
import pickle 
from tkinter import *
from sklearn.feature_extraction.text import TfidfVectorizer
#import preprocess_kgptalkie as ps
import re
import gradio as gr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
'''
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x
'''

'\ndef get_clean(x):\n    x = str(x).lower().replace(\'\\\', \'\').replace(\'_\', \' \')\n    x = ps.cont_exp(x)\n    x = ps.remove_emails(x)\n    x = ps.remove_urls(x)\n    x = ps.remove_html_tags(x)\n    x = ps.remove_accented_chars(x)\n    x = ps.remove_special_chars(x)\n    x = re.sub("(.)\\1{2,}", "\\1", x)\n    return x\n'

In [5]:
def get_clean(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [6]:
tfidf = pickle.load(open("../content/vectorizer.pickle", 'rb'))
loaded_model_linearsvc = pickle.load(open('../content/model', 'rb'))
loaded_model_xgb = pickle.load(open('../content/model_xgb', 'rb'))
loaded_model_rf = pickle.load(open('../content/model_rf', 'rb'))

In [7]:
def pre_processing(x):
  x = get_clean(x)
  vec = tfidf.transform([x])
  return vec

In [8]:
def classify(num):
  if num<0.5:
    return 'Neutral'
  elif num>=0.5 and num<=1.5:
    return 'Positive'
  else:
    return 'Negative'

In [9]:
#def average(outputs):
#  return sum(outputs) / len(outputs)

#average(outputs)

In [10]:
def predict(text_to_analyse):
  #outputs = []
  x = text_to_analyse
  vec = pre_processing(x)
  LSVC = loaded_model_linearsvc.predict(vec)
  output_svc = classify(LSVC)
  #outputs.append(num)
  XGB  = loaded_model_xgb.predict(vec)
  output_xgb = classify(XGB)
  #outputs.append(num)
  rf = loaded_model_rf.predict(vec)
  output_rf = classify(rf)
  #outputs.append(num)
  output1 = "The output of LinearSVC is " + output_svc
  output2 = "The output of XGBoost is " + output_xgb
  output3 = "The output of Random forest is " + output_rf
  return (output1, output2, output3)

In [11]:
interface = gr.Interface(
    fn = predict,
    inputs = ['text'],
    outputs =  ['text','text','text'],
    title="TEXT ANALYSIS",
    live = True
)

In [12]:
interface.launch(debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://53388.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f9c2e380b50>,
 'http://127.0.0.1:7860/',
 'https://53388.gradio.app')

**Code for testing purpose and getting predictions from csv on multiple inputs.** Only run this after uploading the csv

In [13]:
import pandas as pd

df = pd.read_csv("/content/Test_data.csv")   #input the csv file and copy the location only then run this.
df['Input'] = df['Input'].apply(lambda x: get_clean(x))
Input_list = df['Input'].tolist()
for i in Input_list: 
  x = i
  vec = tfidf.transform([x])
  vec.shape
  XGB  = loaded_model_xgb.predict(vec)       #Can change the model 
  output_xgb = classify(XGB)          
  print(output_xgb)
  

Positive
Neutral
Neutral
Negative
Neutral
Positive
Negative
Neutral
Positive
Neutral
